In [1]:
import os

In [2]:

%pwd

'd:\\Wine_Quality_Prediction_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Wine_Quality_Prediction_Project'

**4.Prepare the Entity here and then Update it in modular programming**

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

**5.Prepare the configuration manager in src config and then Update the configuration manager in modular programming**


In [ ]:

from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForest
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = Path("artifacts/model_trainer/randomForest.joblib"),
            all_params=params,
            metric_file_name = Path("artifacts/model_evaluation/rf-metrics.json"),
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/entbappy/End-to-end-ML-Project-with-DVC-MLflow.mlflow",
           
        )

        return model_evaluation_config

**6.Prepare the Model Evaluation components with MlFlow Logging and then Update the components in modular coding**

**Connect with Dagshub repository**

In [8]:
import dagshub
dagshub.init(repo_owner='wasimshag', repo_name='Wine_Quality_Prediction_Project_ML', mlflow=True)


[2024-12-08 20:20:07,453: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as wasimshag

[2024-12-08 20:20:07,477: INFO: helpers: Accessing as wasimshag]
[2024-12-08 20:20:08,811: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/wasimshag/Wine_Quality_Prediction_Project_ML "HTTP/1.1 200 OK"]
[2024-12-08 20:20:10,026: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "wasimshag/Wine_Quality_Prediction_Project_ML"

[2024-12-08 20:20:10,029: INFO: helpers: Initialized MLflow to track repo "wasimshag/Wine_Quality_Prediction_Project_ML"]


Repository wasimshag/Wine_Quality_Prediction_Project_ML initialized!

[2024-12-08 20:20:10,032: INFO: helpers: Repository wasimshag/Wine_Quality_Prediction_Project_ML initialized!]


In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="RFModel")
            else:
                mlflow.sklearn.log_model(model, "model")

            

**7. Create the Pipeline and then Update the pipeline in Modular coding**

In [ ]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e